# Confiabilidade de viga mista

## Importando bibliotecas

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import import_ipynb
from math import sqrt
import pandas as pd

## Tabela de perfis

In [ ]:
tabela = pd.read_excel('Catalogo-Tabela-Bitolas-Editado-2.xlsx')
display(tabela)

## Definindo funções

In [ ]:
def calculo_destribuicao_media(media: float, desvpad: float, area: bool = False) -> list:
    AMOSTRAS: int = 1000
    medias: list = []
    areas: list = []
    
    if (area):
        for i in range(AMOSTRAS):
            numero_aleatorio: float = np.random.rand()
            medias.append(norm.ppf(numero_aleatorio, media, desvpad))
            areas.append(np.pi * (medias[i] / 2) ** 2)
        
        return medias, areas

    else:
        for i in range(AMOSTRAS):
            numero_aleatorio: float = np.random.rand()
            medias.append(norm.ppf(numero_aleatorio, media, desvpad))
            
        return medias
        

## Médias - Espessura da laje

In [ ]:
espessura_laje: float = 10 # cm 
desvpad_espessura_laje: float = 0.06 * espessura_laje # cm

espessuras_laje: list = calculo_destribuicao_media(espessura_laje, desvpad_espessura_laje)

## Médias - Limite de escoamento do perfil de aço

In [ ]:
limite_escoamento_perfil: float = 250 * 1.08 # MPa
desvpad_escoamento_perfil: float = 0.08 * limite_escoamento_perfil # MPa

limites_escoamento_perfil: list = calculo_destribuicao_media(limite_escoamento_perfil, desvpad_escoamento_perfil)

## Médias - Limite de escoamento da armadura

In [ ]:
limite_escoamento_armadura: float = 500 * 1.08 # MPa
desvpad_escoamento_armadura: float = 0.05 * limite_escoamento_armadura #MPa

limites_escoamento_armadura: list = calculo_destribuicao_media(limite_escoamento_armadura, desvpad_escoamento_armadura)

## Médias - Resistência do concreto à compressão

In [ ]:
resistencia_concreto: float = 20 * 1.17 # MPa
desvpad_resistencia_concreto: float = 0.15 * resistencia_concreto # MPa

resistencias_concreto: list = calculo_destribuicao_media(resistencia_concreto, desvpad_resistencia_concreto)

## Médias - Diamêtro do rebite

In [ ]:
diametro_rebite: float = 1.59 # cm
desvpad_diametro_rebite: float = 0.01 * diametro_rebite # cm

diametros_rebites, areas_rebites = calculo_destribuicao_media(diametro_rebite, desvpad_diametro_rebite, True)


## Médias - Resistência do rebite

In [ ]:
resistencia_rebite: float = 415 * 1.06 # MPa
desvpad_resistencia_rebite: float = 0.06 * resistencia_rebite # MPa

resistencias_rebite: list = calculo_destribuicao_media(resistencia_rebite, desvpad_resistencia_rebite)

## Pré-dimensionamento

In [ ]:
carga_permanente_g1: float = 7.6 # kN/m
carga_permanente_g2: float = 5 # kN/m

variacao_carga: list = [3, 11]
carga_utilizacao_q2: list = [i for i in range(variacao_carga[0], variacao_carga[1])]

carga_construcao_q1: list = [i / 5.6 for i in carga_utilizacao_q2]

# Propriedades da laje
vao_laje: float = 9 # m
espacamento_laje: float = 2.8 # m

# Carga distrubuída do projeto
carga_distribuida_projeto: list = [1.4 * (carga_permanente_g1 + carga_permanente_g2) + carga_construcao_q1[i] * carga_utilizacao_q2[i] for i in range(len(carga_utilizacao_q2))]

# Momento solicitante do projeto
momento_solicitante_projeto: list = [(carga_distribuida_projeto[i] * (vao_laje ** 2)) / 8 for i in range(len(carga_distribuida_projeto))]

altura_perfil_ht: float = 45 # cm
coeficiente_ya1: float = 1.1
x: float = 10 # cm

# Tabela com áreas filtradas a partir de 450 mm
tabela_filtrada = tabela.loc[tabela['d'] >= 450]

row_tabela: int = 0

# Área mínima da seção de aço
areas_minimas: list = [[] for i in range(len(momento_solicitante_projeto))]
index_matriz: int = 0
area_minima_inicial: float = tabela_filtrada.iloc[0]['area']
for i in momento_solicitante_projeto:
    for j in range(len(limites_escoamento_perfil)):
        areas_minimas[index_matriz].append((i * 100) / (((limites_escoamento_perfil[j] / 10) / coeficiente_ya1) * ((altura_perfil_ht / 2) + espessuras_laje[j] - (x / 2)))) # cm²
        if (areas_minimas[index_matriz][j] > area_minima_inicial):
            print(f'Área mínima inicial: {area_minima_inicial}')
            print(f'Área mínima calculada: {areas_minimas[index_matriz][j]}')
            row_tabela += 1
            area_minima_inicial = tabela_filtrada.iloc[row_tabela]['area']
    index_matriz += 1
    

print(areas_minimas)
print(f'Área mínima final {area_minima_inicial}')


## Flambagem da seção

In [ ]:
# Classificação da seção quanto a flambagem
altura_perfil: float = tabela_filtrada.iloc[row_tabela]['d\''] # mm
espessura_perfil: float = tabela_filtrada.iloc[row_tabela]['tw'] # mm
print(altura_perfil, espessura_perfil)
modulo_elasticidade_perfil: float = 200000 # MPa

esbeltez_alma: float = altura_perfil / espessura_perfil

limites_flambagem_local: list = []

def calculo_limite_flambagem_local(coef: float, modulo_elasticidade_perfil: float, limite_escoamento_perfil: float) -> float:
    return coef * sqrt(modulo_elasticidade_perfil / limite_escoamento_perfil)

countador_compacto: int = 0
countador_semicompacto: int = 0

for i in range(len(limites_escoamento_perfil)):
    limite = calculo_limite_flambagem_local(3.76, modulo_elasticidade_perfil, limites_escoamento_perfil[i])
    limites_flambagem_local.append(limite)
    
    if esbeltez_alma <= limite:
        countador_compacto += 1
    elif esbeltez_alma > limite and esbeltez_alma < calculo_limite_flambagem_local(5.7, modulo_elasticidade_perfil, limites_escoamento_perfil[i]):
        countador_semicompacto += 1
    else:
        print('Erro')
        
print(f'Compactas: {countador_compacto}')
print(f'Semi-compactas: {countador_semicompacto}')
        

## Momento resistência da viga mista